In [1]:

import plotly.graph_objs as go
import plotly.io as pio
import pandas as pd

In [2]:
df = pd.read_csv('/Users/joaosequeira/udacity_joao/atp_analysis/atp_tennis.csv')


In [3]:
df.head(10)

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
0,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mayer F.,Giraldo S.,Mayer F.,28,57,1215,778,1.36,3.00,6-4 6-4
1,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,Nieminen J.,Nieminen J.,35,41,1075,927,2.20,1.61,3-6 6-2 1-6
2,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Nishikori K.,Matosevic M.,Nishikori K.,19,49,1830,845,1.25,3.75,7-5 6-2
3,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mitchell B.,Baghdatis M.,Baghdatis M.,326,36,137,1070,9.00,1.07,4-6 4-6
4,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,Klizan M.,Istomin D.,43,30,897,1175,1.90,1.80,6-1 6-2
5,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Ito T.,Millman J.,Millman J.,79,199,655,239,2.20,1.61,4-6 1-6
6,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Falla A.,Levine J.,Falla A.,54,104,809,530,2.20,1.61,6-1 7-6
7,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Kudla D.,Melzer J.,Melzer J.,137,29,402,1177,2.62,1.44,6-2 4-6 4-6
8,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Robredo T.,Harrison R.,Robredo T.,114,69,495,710,3.00,1.36,6-4 7-6
9,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Baker B.,Dimitrov G.,Dimitrov G.,61,48,756,866,3.00,1.36,3-6 6-7


In [60]:


def winner_metrics(col_x, col_y, df=df, winner_col='Winner', player1_col='Player_1', player2_col='Player_2'):
    """
    Função que extrai valores de colunas de um DataFrame, de acordo com condições de outras colunas.

    Argumentos:
    df -- DataFrame Pandas a ser utilizado.
    winner_col -- Nome da coluna que será comparada com "player1_col" e "player2_col".
    player1_col -- Nome da coluna que será comparada com "winner_col".
    player2_col -- Nome da coluna que será comparada com "winner_col".
    col_x -- Nome da coluna de onde será extraído o valor caso "player1_col=winner_col".
    col_y -- Nome da coluna de onde será extraído o valor caso "player2_col=winner_col".

    Retorna:
    Um DataFrame com as colunas "winner" e "odd".
    """

    # Inicializa dicionários para armazenar os valores extraídos
    result_dict = {"winner": [], "metric": []}

    # Itera sobre as linhas do DataFrame
    for index, row in df.iterrows():

        # Compara os valores das colunas "winner_col" e "player1_col"
        if row[player1_col] == row[winner_col]:
            result_dict["winner"].append(row[player1_col])
            result_dict["metric"].append(row[col_x])

        # Compara os valores das colunas "winner_col" e "player2_col"
        elif row[player2_col] == row[winner_col]:
            result_dict["winner"].append(row[player2_col])
            result_dict["metric"].append(row[col_y])

    # Retorna um DataFrame com as colunas "winner" e "odd"
    return pd.DataFrame(result_dict)


In [83]:
df_winner_odds = winner_metrics('Odd_1','Odd_2')

counts = df_winner_odds['winner'].value_counts()

top_counts = counts.sort_values(ascending=False)[:15]

treated_odd = [odd for odd in df_winner_odds['metric'] if odd <= 6]



fig = go.Figure(data=[go.Bar(

            x=top_counts.index, 
            y=top_counts.values

)])

fig.update_layout(title="Quantidade partidas ganhas")

pio.show(fig)


# Cria o histograma com a função "histogram"
fig = go.Figure(data=[go.Histogram(x=treated_odd, nbinsx=10)])

fig.update_layout(title="Distribuição de odds")

# Exibe o gráfico
fig.show()


In [78]:
# Define uma função que retorna o vencedor de uma partida
def get_winner_rank(df):
    if df['Winner'] == df['Player_1']:
        return df['Rank_1']
    else:
        return df['Rank_2']
    
# Define uma função que retorna o perdedor de uma partida
def get_loser_rank(df):
    if df['Winner'] != df['Player_1']:
        return df['Rank_1']
    else:
        return df['Rank_2']
    
# Aplica a função ao DataFrame para criar a coluna "Loser"
df['Rank_loser'] = df.apply(get_loser_rank, axis=1)
df['Rank_winner'] = df.apply(get_winner_rank, axis=1)

# Criando df com os ganhadores tendo rank pior
df_loser_bigger_rank = df[df['Rank_winner'] > df['Rank_loser']]

count_loser_bigger_rank = len(df_loser_bigger_rank)/len(df)
print("Número de eventos onde o rank do vencedor é maior do que o do perdedor: ", count_loser_bigger_rank)



Número de eventos onde o rank do vencedor é maior do que o do perdedor:  0.351391846068922


In [76]:




counts = df_loser_bigger_rank['winner'].value_counts()

top_counts = counts.sort_values(ascending=False)[:15]


# Cria o histograma com a função "histogram"
fig = go.Figure(data=[go.Histogram(x=treated_odd, nbinsx=10)])

fig.update_layout(title="Distribuição de odds")

# Exibe o gráfico
fig.show()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score,Loser,Rank_loser,Rank_winner
1,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,Nieminen J.,Nieminen J.,35,41,1075,927,2.20,1.61,3-6 6-2 1-6,Benneteau J.,35,41
4,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,Klizan M.,Istomin D.,43,30,897,1175,1.90,1.80,6-1 6-2,Klizan M.,30,43
5,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Ito T.,Millman J.,Millman J.,79,199,655,239,2.20,1.61,4-6 1-6,Ito T.,79,199
8,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Robredo T.,Harrison R.,Robredo T.,114,69,495,710,3.00,1.36,6-4 7-6,Harrison R.,69,114
13,Brisbane International,2013-01-02,ATP250,Outdoor,Hard,2nd Round,3,Mayer F.,Baghdatis M.,Baghdatis M.,28,36,1215,1070,2.62,1.44,4-6 2-6,Mayer F.,28,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25348,Miami Open,2023-03-28,Masters 1000,Outdoor,Hard,4th Round,3,Fritz T.,Rune H.,Fritz T.,10,8,2975,3325,1.62,2.30,6-3 6-4,Rune H.,8,10
25349,Miami Open,2023-03-28,Masters 1000,Outdoor,Hard,4th Round,3,Van De Zandschulp B.,Ruusuvuori E.,Ruusuvuori E.,32,54,1170,866,2.20,1.67,6-4 4-6 5-7,Van De Zandschulp B.,32,54
25351,Miami Open,2023-03-28,Masters 1000,Outdoor,Hard,4th Round,3,Tsitsipas S.,Khachanov K.,Khachanov K.,3,16,5770,2505,1.91,1.91,6-7 4-6,Tsitsipas S.,3,16
25352,Miami Open,2023-03-28,Masters 1000,Outdoor,Hard,4th Round,3,Eubanks C.,Mannarino A.,Eubanks C.,119,62,479,826,2.50,1.53,7-6 7-6,Mannarino A.,62,119
